In [2]:
#I will be sending MIDI notes

#channel variables for the different instruments to send the midi notes to Cubase
CH_KICK = 0 #channel 0: kick drum
CH_SNARE = 1 #channel 1: snare drum
CH_HIHAT = 2 #channel 2: hihat
CH_BASS = 3 #channel 3: bass
CH_CHORDS = 4 #channel 4: chords
CH_MELODY = 5 #channel 5: melody
#channels array (13 elements) according to data array 
ch_arr = [CH_MELODY, CH_BASS, CH_CHORDS, CH_CHORDS, CH_CHORDS, CH_CHORDS, CH_CHORDS, CH_CHORDS, CH_CHORDS, CH_CHORDS, CH_SNARE, CH_HIHAT, CH_KICK]

#(midiOutput.note_on(noteval, noteVel) #send the midi note to the Launchpad)
global preNoteMelody #global variable to store the previous note played by the melody
preNoteMelody = 0

#empty array of the data to be sent
data = [0 for i in range(13)]

#function that converts index to midi note (scale)
#for example, 1->60, 2->62, ...
scale = [0,2,3,5,7,8,10,12,14,15,17,18,20,22] #minor scale
def indexToMidi(n):
    return scale[n]+30

#function that converts a note to midi value and adds another offset to transpose if wanted.The first offset is used, for example, to make the bass an octave lower, and the second offset is used to change the key/scale.

offset2 = 0 #global variable for offset2

def noteToMidi(n,offset1):
    global offset2
    if n == -1:
        return -1
    else:
        return indexToMidi(n)+offset1+offset2

#function that prepares the snare and hihat
# - if -1, return 0
# - else, return 10
def stepPerc(n):
    if n == -1:
        return -1
    elif n==0:
        return 36 #C2
    else:
        return 43 #G2

#function that that fills the data array with the actual data
def sendMusic(chord, bass, melody, snare, hihat, counter, cycle):
    cycle2 = 1 if cycle>1 else 0 #cycle2 is a variable that activates every 2 cycles of 16 steps. 
    #cycle is modulo 4, so that it can take values 0,1,2,3. If it is 2 or 3, then cycle2 is 1.

    #first the melody (q0) and the bass
    data[0] = noteToMidi(melody,24)
    data[1] = noteToMidi(bass[cycle2], 0) if counter==0 else -1

    #now chord notes (0-7)
    for i in range(8):
        data[i+2] = noteToMidi(i,24) if (counter==0 and chord[cycle2][i]) else -1

    #now snare and hihat
    data[10] = stepPerc(snare)
    data[11] = stepPerc(hihat)

    #kick drum signal to trigger it in Pd every 4 steps
    data[12] = 10 if counter%4==0 else -1

    #data to str to print on the console
    str1 = ""
    for i in range(13):
        str1 += str(data[i]) + " "
    #log(str1)

    #create a loop to send note off (vel = 0) to all chord notes after being pressed
    if counter == 15:
        for i in range(8):
            noteVal = noteToMidi(i,24)
            noteVel = 0
            noteCh = CH_CHORDS
            midiOutputCUB.note_on(noteVal, noteVel, noteCh)
    

    #data to midi to be sent to Cubase
    global preNoteMelody
    # Turn off previous note
    midiOutputCUB.note_on(preNoteMelody, 0, ch_arr[0]) #(note, velocity, channel)
    for i in range(13):
        if i == 0:
            preNoteMelody = data[i]
        if data[i] != -1:
            #log("note on: " + str(data[i]) + " channel: " + str(ch_arr[i]))
            midiOutputCUB.note_on(data[i], 100, ch_arr[i]) #(note, velocity, channel)